<a href="https://colab.research.google.com/github/ict3104-p2-team08/ict3104-p2-team08/blob/section-dataset/3104_T08.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Activity Detection ML Test

In [ ]:
#Add into console if colab keeps disconnecting
function KeepClicking(){
   console.log("Clicking");
   document.querySelector("colab-toolbar-button#connect").click()
}setInterval(KeepClicking,600,000)

# **TSU PIPLINE**


---



## 1. Prerequisites

This section provides the setting up of the environment

In [1]:
!pip install torch -f https://download.pytorch.org/whl/cu113/torch-1.12.1%2Bcu113-cp37-cp37m-linux_x86_64.whl
!pip install torchaudio -f https://download.pytorch.org/whl/cu113/torchaudio-0.12.1%2Bcu113-cp37-cp37m-linux_x86_64.whl
!pip install torchsummary==1.5.1 torchtext==0.13.1 torchvision -f https://download.pytorch.org/whl/cu113/torchvision-0.13.1%2Bcu113-cp37-cp37m-linux_x86_64.whl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/cu113/torch-1.12.1%2Bcu113-cp37-cp37m-linux_x86_64.whl
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/cu113/torchaudio-0.12.1%2Bcu113-cp37-cp37m-linux_x86_64.whl
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/cu113/torchvision-0.13.1%2Bcu113-cp37-cp37m-linux_x86_64.whl


In [2]:
!git clone https://github.com/v-iashin/video_features.git
!pip install omegaconf==2.0.6

Cloning into 'video_features'...
remote: Enumerating objects: 1283, done.
remote: Counting objects: 100% (381/381), done.
remote: Compressing objects: 100% (165/165), done.
remote: Total 1283 (delta 245), reused 295 (delta 200), pack-reused 902
Receiving objects: 100% (1283/1283), 288.54 MiB | 35.90 MiB/s, done.
Resolving deltas: 100% (664/664), done.
Checking out files: 100% (177/177), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
# Imports widgets for jupyter
try:
    import piplite
    piplite.install(['ipywidgets'])
except ImportError:
    pass

## 2. Toggling of pipelines

This section shows the toggling of dependencies between TSU and STEP

In [ ]:
#insert code here

## 3. Data exploration

This section load and display video data from the Toyota Smarthome (TSU) project

In [6]:
#import our github files
!git clone https://github.com/ict3104-p2-team08/ict3104-p2-team08.git

Cloning into 'ict3104-p2-team08'...
remote: Enumerating objects: 86, done.
remote: Counting objects: 100% (25/25), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 86 (delta 10), reused 8 (delta 3), pack-reused 61
Unpacking objects: 100% (86/86), done.
Checking out files: 100% (25/25), done.


In [ ]:
#tsu video files to display in dropdown
from ipywidgets import  Dropdown, widgets

w = widgets.Dropdown(
    options= file_list,
)

def on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        print(change['new'])

w.observe(on_change)

display(w)

In [ ]:
#display video 
from IPython.display import HTML
from base64 import b64encode

video_path = '/content/data/input_files/%' + w.value

locate= "" 

for i in video_path: 
   if i=='%': 
       pass
   else: 
       locate+=i 

def show_video(locate, video_width = 500):
  video_file = open(locate, "r+b").read()
  video_url = f"data:video/mp4;base64,{b64encode(video_file).decode()}"
  return HTML(f"""<video width={video_width} controls><source src="{video_url}"></video>""")
  
show_video(locate)

In [ ]:
#only run it for the first time
%cd video_features

In [ ]:
#extract to I3D feature file
from models.i3d.extract_i3d import ExtractI3D
from utils.utils import build_cfg_path
from omegaconf import OmegaConf
import torch

# Select the feature type
feature_type = 'i3d'
args = OmegaConf.load(build_cfg_path(feature_type))
args.video_paths = ['/content/data/input_files/P02T02C06.mp4']
args.flow_type = 'raft'

extractor = ExtractI3D(args)
print(locate)

# Extract features
for video_path in args.video_paths:
    print(f'Extracting for {video_path}')
    feature_dict = extractor.extract(video_path)
    [(print(k), print(v.shape), print(v)) for k, v in feature_dict.items()]

## 4. Inference with pretrained model

This section shows

In [ ]:
#insert codes here

## 5. Testing and evaluating trained models

This section shows

In [ ]:
#insert codes here

## 6. Training of new model

This section shows

# **STEP PIPLINE**



---



In [ ]:
#insert codes here